In [1]:
# Instalação das dependências (execute se necessário)
!pip install requests beautifulsoup4 pandas numpy pyarrow

# %% [code]
import requests
import pandas as pd
import json
import base64
from datetime import datetime
import io
import numpy as np
from bs4 import BeautifulSoup, Tag
from unicodedata import normalize, combining
from re import sub

# ===============================
# Constantes e Configurações
# ===============================
LINK_API_IBOV = r'https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/'
SEGMENTO_CONSULTAR_POR_CODIGO = 1
SEGMENTO_CONSULTAR_POR_SETOR_ATUACAO = 2

# ===============================
# Classe de Utilidades
# ===============================
class MLET4:
    def __init__(self) -> None:
        pass

    def normalizarTexto(self, pString: str, pSubstituirEspaco: str = '_') -> str:
        vStringNormalizada = normalize('NFKD', pString)
        vStringNormalizada = sub('[^A-Za-z0-9_ ]','', vStringNormalizada)
        vStringNormalizada = vStringNormalizada.strip().upper().replace(' ', '><').replace('<>', '').replace('><', pSubstituirEspaco)
        vStringNormalizada = ''.join([c for c in vStringNormalizada if not combining(c)])
        return vStringNormalizada

# ===============================
# Funções Auxiliares
# ===============================
def payload_2_base64(pPayload_Dict: dict) -> str:
    vPayload_Json = json.dumps(pPayload_Dict)
    vPayload_base64 = base64.b64encode(vPayload_Json.encode()).decode()
    return vPayload_base64

def empresas_IBOV(pLink_API: str, pNumeroPagina: int = 1, pQtd_Itens_Pagina: int = 100,
                  pSegmentoEmpresas: int = SEGMENTO_CONSULTAR_POR_SETOR_ATUACAO) -> tuple:
    vPayload = {
        'language': 'pt-br',
        'pageNumber': pNumeroPagina,
        'pageSize': pQtd_Itens_Pagina,
        'index': 'IBOV',
        'segment': pSegmentoEmpresas
    }
    vLinkAPI = ''.join([pLink_API, payload_2_base64(vPayload)])
    vRequisicao = requests.get(vLinkAPI)
    json_resp = vRequisicao.json()
    vCabecalho_Json = json_resp.get("header", [])
    vEmpresas_Json = json_resp.get("results", [])
    vTotalPaginas = json_resp['page']['totalPages']
    return (vTotalPaginas, vCabecalho_Json, vEmpresas_Json)

def salvar_localmente(pDataFrame: pd.DataFrame, nome_arquivo: str):
    pDataFrame.to_parquet(nome_arquivo, index=False)
    print(f"Arquivo salvo localmente como {nome_arquivo}")

# ===============================
# Fluxo Principal de Execução
# ===============================
u = MLET4()

vListaEmpresas_Json = []
vContadorPagina = 1
vTotalPaginas = 1

# Loop de extração paginada da API
while vContadorPagina <= vTotalPaginas:
    vResultadoEmpresas = empresas_IBOV(pLink_API=LINK_API_IBOV, pNumeroPagina=vContadorPagina,
                                       pQtd_Itens_Pagina=100, pSegmentoEmpresas=SEGMENTO_CONSULTAR_POR_SETOR_ATUACAO)
    vTotalPaginas = vResultadoEmpresas[0]
    vCabecalhoEmpresas_Json = vResultadoEmpresas[1]
    vListaEmpresas_Json.extend(vResultadoEmpresas[2])
    vContadorPagina += 1

# Se houver dados, realiza transformação
if vListaEmpresas_Json:
    vBase = pd.DataFrame(vListaEmpresas_Json)

    # Adiciona informações do cabeçalho
    vBase['dt_referencia_carteira'] = int(pd.to_datetime(vCabecalhoEmpresas_Json['date'], format='%d/%m/%y').strftime('%Y%m%d'))
    vBase['qtd_teorica_total'] = pd.to_numeric(vCabecalhoEmpresas_Json['theoricalQty'].replace('.', '').replace(',', '.'))
    vBase['qtd_redutor'] = pd.to_numeric(vCabecalhoEmpresas_Json['reductor'].replace('.', '').replace(',', '.'))

    # ⚠️ Corrigido: Converte DT_EXTRACAO para um formato compatível com AWS Glue
    vBase['dt_extracao'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Converte para string

    # Renomeia e formata as colunas
    vBase.rename(columns={
        'segment': 'nme_setor',
        'cod': 'cod_empresa',
        'asset': 'nme_acao',
        'type': 'dsc_tipo',
        'part': 'val_participacao_setor',
        'partAcum': 'val_participacao_acumulada_setor',
        'theoricalQty': 'qtd_teorica'
    }, inplace=True)

    # Aplica normalização nos nomes das colunas
    vBase.columns = [u.normalizarTexto(i) for i in vBase.columns]

    # Converte colunas numéricas que estão em formato de string
    colunas_num = ['VAL_PARTICIPACAO_SETOR', 'VAL_PARTICIPACAO_ACUMULADA_SETOR', 'QTD_TEORICA']
    vBase[colunas_num] = (vBase[colunas_num]
                          .apply(lambda x: x.str.replace('.', '').str.replace(',', '.'))
                          .apply(pd.to_numeric))

    # Remove excesso de espaços em colunas do tipo string
    vBase = vBase.apply(lambda x: x.str.replace(r'\s+', ' ', regex=True).str.strip() if x.dtype == 'object' else x)

    # Salva o arquivo Parquet localmente
    nome_arquivo = f"Empresas_IBOV_{vBase['DT_REFERENCIA_CARTEIRA'].max()}_{vBase['DT_EXTRACAO'].max().replace(':', '').replace('-', '').replace(' ', '_')}.parquet"
    salvar_localmente(vBase, nome_arquivo)
else:
    print("Nenhum dado retornado pela API.")

Arquivo salvo localmente como Empresas_IBOV_20250305_20250305_211156.parquet


In [2]:
# Exibir as 5 primeiras linhas
print('=' * 200)
vBase = pd.read_parquet(nome_arquivo)
vBase.head()

,NME_SETOR,COD_EMPRESA,NME_ACAO,DSC_TIPO,VAL_PARTICIPACAO_SETOR,VAL_PARTICIPACAO_ACUMULADA_SETOR,QTD_TEORICA,DT_REFERENCIA_CARTEIRA,QTD_TEORICA_TOTAL,QTD_REDUTOR,DT_EXTRACAO
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON ED NM,3.014,3.014,1243177587,20250305,98454079581,1.616393e+07,2025-03-05 21:11:56
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,2.580,2.827,734631801,20250305,98454079581,1.616393e+07,2025-03-05 21:11:56
2,Bens Indls / Mat Transporte,POMO4,MARCOPOLO,PN ED N2,0.247,2.827,666378439,20250305,98454079581,1.616393e+07,2025-03-05 21:11:56
3,Bens Indls/Transporte,AZUL4,AZUL,PN ES N2,0.064,2.263,326872005,20250305,98454079581,1.616393e+07,2025-03-05 21:11:56
4,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,0.585,2.263,991920937,20250305,98454079581,1.616393e+07,2025-03-05 21:11:56


In [3]:
vBase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NME_SETOR                         87 non-null     object 
 1   COD_EMPRESA                       87 non-null     object 
 2   NME_ACAO                          87 non-null     object 
 3   DSC_TIPO                          87 non-null     object 
 4   VAL_PARTICIPACAO_SETOR            87 non-null     float64
 5   VAL_PARTICIPACAO_ACUMULADA_SETOR  87 non-null     float64
 6   QTD_TEORICA                       87 non-null     int64  
 7   DT_REFERENCIA_CARTEIRA            87 non-null     int64  
 8   QTD_TEORICA_TOTAL                 87 non-null     int64  
 9   QTD_REDUTOR                       87 non-null     float64
 10  DT_EXTRACAO                       87 non-null     object 
dtypes: float64(3), int64(3), object(5)
memory usage: 7.6+ KB


In [4]:
vBase.to_csv('Empresas_IBOV.csv', index=False)